In [ ]:
import os
import pycocotools
os.getcwd()


sample_data


In [ ]:
%%bash
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
import numpy as np
import torch
import torch.utils.data
from PIL import Image
from PIL import ImageDraw
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import torchvision
import xml.etree.ElementTree as ET
import glob
#import nye funksjonen vår

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
pathDataset = 'drive/MyDrive/Baggages'
# bytt ut med nye 
#imgbbox = load_dataset(pathDataset, deleteFiles=False)



In [ ]:
class GunDataset(torch.utils.data.Dataset):
    
    def __init__(self, dicPics,categories, path, transforms=None): 
        self.path = path
        self.dicPics = dicPics
        self.transforms = transforms
        self.categories = categories
        self.imgs = [o for o in imgbbox]

    def __getitem__(self, idx):
        img = Image.open(self.path +"/"+ self.imgs[idx]).convert("RGB")        
        box_list = self.dicPics[self.imgs[idx]][0]
        
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
        labels_list =  self.dicPics[self.imgs[idx]][1]

        # there is only one class
        #labels = torch.ones((num_objs,), dtype=torch.int64) 

        # multible classes
        labels = torch.zeros((num_objs,), dtype=torch.int64)

        for i in range(num_objs):
          labels[i] = self.categories[labels_list[i]]
        #print(labels)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])

        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        return img, target
    
    def __len__(self):
        return len(self.imgs)

In [ ]:
cat = {'handgun': 1,'knife': 2, 'razorblade': 3, 'shuriken': 4}
dataset = GunDataset(dicPics = imgbbox,categories = cat, path = pathDataset, transforms = None)
dataset.__getitem__(10)

In [ ]:
testimg = 'B0015_0001.png'
image = Image.open(pathDataset +"/"+ testimg)
image

In [ ]:
def get_model(num_classes):
  # load an object detection model pre-trained on COCO
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get the number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new on
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
   
  return model

In [ ]:
def get_transform(train):
    transforms = []
   # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
      # during training, randomly flip the training images
      # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
# use our dataset and defined transformations
dataset = GunDataset(dicPics=imgbbox, categories = cat, path = pathDataset, transforms = get_transform(train=True))         # Training
dataset_test = GunDataset(dicPics = imgbbox, categories = cat, path = pathDataset, transforms = get_transform(train=False)) # Testing

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-40])  
#dataset = torch.utils.data.Subset(dataset, indices[:100])  # testing

dataset_test = torch.utils.data.Subset(dataset_test, indices[-40:])
#dataset_test = torch.utils.data.Subset(dataset_test, indices[-30:])


# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
              dataset, batch_size=2, shuffle=True, num_workers=4,
              collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
         dataset_test, batch_size=1, shuffle=False, num_workers=4,
         collate_fn=utils.collate_fn)

print("We have: {} examples, {} are training and {} testing".format(len(indices), len(dataset), len(dataset_test)))

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - gun and not gun
num_classes = 3
# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochs

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# TRAIN NEW MODEL

In [ ]:
num_epochs = 0
for epoch in range(num_epochs):
   # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
# update the learning rate

    lr_scheduler.step()
   # evaluate on the test dataset
   # evaluate(model, data_loader_test, device=device)

In [ ]:
 # Saves model to folder trainedModels/XRAY
    
path_trnd_model = "drive/MyDrive/trainedModels/XRAY"
model_name = "model_"+str(len(cat))+"Classes_"+str(num_epochs)+"epochs"

if os.path.isdir(path_trnd_model) is False:
    os.mkdir(path_trnd_model)

state = {'epoch': num_epochs +1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict() }
torch.save(state, path_trnd_model+"/"+model_na,e)

#torch.save(model.state_dict(), )

#TRAIN OLD MODEL

In [ ]:
def load_checkpoint(model, optimizer=None, filename=None):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        if optimizer is not None:
          optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})".format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch

Name and path of old model

In [ ]:
path_trnd_model = "drive/MyDrive/trainedModels"
# model_numClasses_(num trained epochs)epochs
model_name = "model_5Classes_10epochs"

In [ ]:
model, optimizer, start_epoch = load_checkpoint(get_model(num_classes = 3), optimizer, filename=path_trnd_model+"/"+model_name)
model = model.to(device)

# individually transfer the optimizer parts...
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

In [ ]:
# let's train it for 0 epochs
num_epochs = 0
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

    # update the learning rate
    lr_scheduler.step()
   # evaluate on the test dataset
   # evaluate(model, data_loader_test, device=device)

Save model for further training or save it for good

In [ ]:
saveForGood = False

In [ ]:
if saveForGood:
    state = {'epoch': num_epochs +1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict() }
    torch.save(state, path_trnd_model+"/"model_name)
else:
    # Saves the final model, not trainable anymore
    torch.save(model.state_dict(), path_trnd_model+"/"+model_name + "finished")


#PREDICT

In [ ]:
# To load trained model
path_trnd_model = "drive/MyDrive/trainedModels/XRAY"
model_name = "model_5Classes_10epochs_finished"

loaded_model = get_model(num_classes = 3)

if os.path.isfile(path_trnd_model +"/"+ model_name)):
    loaded_model.load_state_dict(torch.load( path_trnd_model +"/"+ model_name))
else:
    print("Wrong path or filename")

In [ ]:

#idx = 1
for idx in range(1):

    img, _ = dataset_test[idx]
    label_boxes = np.array(dataset_test[idx][1]["boxes"])
    #put the model in evaluation mode
    loaded_model.eval()
    with torch.no_grad():
        prediction = loaded_model([img])
    image = Image.fromarray(img.mul(255).permute(1, 2,0).byte().numpy())
    draw = ImageDraw.Draw(image)

    cat_rev = {cat[o]: o for o in cat}
    # draw groundtruth
    for elem in range(len(label_boxes)):
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
        (label_boxes[elem][2], label_boxes[elem][3])], 
        outline ="green", width =3)
        
    for element in range(len(prediction[0]["boxes"])):

        boxes = prediction[0]["boxes"][element].cpu().numpy()
        score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                        decimals= 4)
        if score > 0.7:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], outline ="red", width =3)
            draw.text((boxes[0], boxes[1]), text = str(score)+ " " + cat_rev[np.int(prediction[0]["labels"][element])] , fill="#000")
    display(image)
